### Library

In [388]:
import torch
import torch.optim as optim
import numpy as np
import seaborn as sns
import torchvision
import pandas as pd
from torchvision import transforms, datasets
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary
import matplotlib as plt
from torch.utils.data import DataLoader, SubsetRandomSampler

In [389]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [390]:
data_dir = './Data'

### Metrics

In [391]:
e_pochs = 100
l_r=0.0001
batch_size = 8
criter = nn.CrossEntropyLoss()


### Data processing


In [392]:
dataset = datasets.ImageFolder(data_dir)
dataset.class_to_idx

{'agronyem_repens': 0,
 'ailanto': 1,
 'arrenatereto': 2,
 'artemisia_verlotorum': 3,
 'populus_nigra': 4,
 'rubus_caesius': 5,
 'ulmus_minor': 6}

In [393]:
size = len(dataset)
indices = list(range(size))
np.random.shuffle(indices)

In [394]:
num_class =len(dataset.classes)
num_class

7

In [395]:
class_image_counts = {c: 0 for c in dataset.classes}
for img, label in dataset:
    class_image_counts[dataset.classes[label]] += 1
for c, count in class_image_counts.items():
    print(f"Class '{c}' contains {count} images.")


Class 'agronyem_repens' contains 119 images.
Class 'ailanto' contains 121 images.
Class 'arrenatereto' contains 82 images.
Class 'artemisia_verlotorum' contains 139 images.
Class 'populus_nigra' contains 128 images.
Class 'rubus_caesius' contains 102 images.
Class 'ulmus_minor' contains 91 images.


In [396]:
dir(transforms)

['AugMix',
 'AutoAugment',
 'AutoAugmentPolicy',
 'CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'ElasticTransform',
 'FiveCrop',
 'GaussianBlur',
 'Grayscale',
 'InterpolationMode',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandAugment',
 'RandomAdjustSharpness',
 'RandomAffine',
 'RandomApply',
 'RandomAutocontrast',
 'RandomChoice',
 'RandomCrop',
 'RandomEqualize',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomInvert',
 'RandomOrder',
 'RandomPerspective',
 'RandomPosterize',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSolarize',
 'RandomVerticalFlip',
 'Resize',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 'TrivialAugmentWide',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional_pil',
 '_functional_tensor',
 '_presets',
 'autoaugment',
 'functional',
 'transforms']

In [397]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

In [410]:
data = datasets.ImageFolder(root = data_dir, transform = preprocess)


(tensor([[[-0.9363, -0.6109, -0.0287,  ..., -2.1179, -2.1179, -2.1179],
          [-0.8507, -0.4226,  0.2796,  ..., -2.1179, -2.1179, -2.1179],
          [-0.7479, -0.2171,  0.6049,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [ 0.1083, -0.3883, -0.7650,  ..., -1.8439, -1.7754, -1.6727],
          [ 0.2967, -0.1828, -0.5424,  ..., -1.7583, -1.7583, -1.7240],
          [ 0.3994, -0.0287, -0.3712,  ..., -1.6213, -1.6898, -1.7583]],
 
         [[-0.6001, -0.2850,  0.2927,  ..., -2.0357, -2.0357, -2.0357],
          [-0.5126, -0.0924,  0.6078,  ..., -2.0357, -2.0357, -2.0357],
          [-0.4251,  0.1001,  0.9230,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [ 0.7479,  0.2402, -0.1450,  ..., -1.7556, -1.7381, -1.6681],
          [ 0.9055,  0.4328,  0.0651,  ..., -1.7031, -1.7731, -1.7906],
          [ 0.9930,  0.5728,  0.2402,  ..., -1.6155, -1.7556, -1.8606]],
 
         [[-1.5779, -1.1421, -0.3753,  ..., -1.8044, -1.8044, -1.8044],
          [-1.5430, -1.0027,

In [399]:
test_split_index = int(np.floor(0.2 * size))
test_split_index

156

In [400]:
test_idx  = indices[:test_split_index]
train_idx = indices[test_split_index:]
train_sampler = SubsetRandomSampler(train_idx)
test_sampler  = SubsetRandomSampler(test_idx)

In [401]:
train_loader = DataLoader(dataset=data, shuffle=False, batch_size=8)
     

In [402]:
data.class_to_idx.items()

dict_items([('agronyem_repens', 0), ('ailanto', 1), ('arrenatereto', 2), ('artemisia_verlotorum', 3), ('populus_nigra', 4), ('rubus_caesius', 5), ('ulmus_minor', 6)])

In [403]:
idx_to_class = {v: k for k, v in data.class_to_idx.items()}
def get_class_distribution_loaders(dataloader):
  count_dict = {k:0 for k,v in data.class_to_idx.items()}
  for input, label in dataloader:
      label = label.item()
      label = idx_to_class[label]
      count_dict[str(label)] += 1    
  return count_dict

### Models


In [404]:
dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [405]:
EfficientNet = models.efficientnet_b7(weights= models.EfficientNet)

In [406]:
summary(EfficientNet)

Layer (type:depth-idx)                        Param #
├─Sequential: 1-1                             --
|    └─Conv2dNormActivation: 2-1              --
|    |    └─Conv2d: 3-1                       1,728
|    |    └─BatchNorm2d: 3-2                  128
|    |    └─SiLU: 3-3                         --
|    └─Sequential: 2-2                        --
|    |    └─MBConv: 3-4                       4,944
|    |    └─MBConv: 3-5                       1,992
|    |    └─MBConv: 3-6                       1,992
|    |    └─MBConv: 3-7                       1,992
|    └─Sequential: 2-3                        --
|    |    └─MBConv: 3-8                       21,224
|    |    └─MBConv: 3-9                       38,700
|    |    └─MBConv: 3-10                      38,700
|    |    └─MBConv: 3-11                      38,700
|    |    └─MBConv: 3-12                      38,700
|    |    └─MBConv: 3-13                      38,700
|    |    └─MBConv: 3-14                      38,700
|    └─Sequential: 2

Layer (type:depth-idx)                        Param #
├─Sequential: 1-1                             --
|    └─Conv2dNormActivation: 2-1              --
|    |    └─Conv2d: 3-1                       1,728
|    |    └─BatchNorm2d: 3-2                  128
|    |    └─SiLU: 3-3                         --
|    └─Sequential: 2-2                        --
|    |    └─MBConv: 3-4                       4,944
|    |    └─MBConv: 3-5                       1,992
|    |    └─MBConv: 3-6                       1,992
|    |    └─MBConv: 3-7                       1,992
|    └─Sequential: 2-3                        --
|    |    └─MBConv: 3-8                       21,224
|    |    └─MBConv: 3-9                       38,700
|    |    └─MBConv: 3-10                      38,700
|    |    └─MBConv: 3-11                      38,700
|    |    └─MBConv: 3-12                      38,700
|    |    └─MBConv: 3-13                      38,700
|    |    └─MBConv: 3-14                      38,700
|    └─Sequential: 2

In [407]:

EfficientNet.classifier = nn.Sequential(
    nn.Dropout(p=0.4),
    nn.Linear(in_features=2046, out_features=num_class),
    nn.Softmax(dim=1)
)


In [411]:
train_loader

In [412]:
optimizer = optim.SGD(EfficientNet.parameters(), lr=l_r)
for ep in range(e_pochs):
    for img, label in train_loader:
        batch_size = img.shape[0]
        out = EfficientNet(img.view(batch_size,-1))
        loss = loss(out, torch.tensor([label]))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (ep, float(loss)))


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_21908\1654504871.py", line 3, in <module>
    for img, label in train_loader:
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\utils\data\dataloader.py", line 630, in __next__
    data = self._next_data()
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\utils\data\dataloader.py", line 674, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_